In [ ]:
from pyspark.sql import DataFrame
from pyspark.sql import *
import numpy as np
import pandas as pd
import scipy as sc
import timeit as ti

In [ ]:
df = spark.read.csv('/test.csv', inferSchema=True, header=True)
dfbig= spark.read.json('/review.json')

Hypotethetically starting to test the speeds of different cachce types. Have more planned but got hung up on the compile process hanging and had to fix that issue. 

In [ ]:
#first try without cacheing
dfbig.groupBy(col("stars")).count().collect()

In [ ]:
#this would hypothetically work, if I had the memory available to cahce the data to
dfbig.cache()
dfbig.groupBy(col("stars")).count().collect() #had an output, but then cleverly restarted the kernell

In [ ]:
#persist to disk as opposed to memory
dfbig.persist(DISK_ONLY)
dfbig.groupBy(col("stars")).count().collect()

# [Persist vs. cache](https://stackoverflow.com/questions/26870537/what-is-the-difference-between-cache-and-persist)
- Persist: can store in other areas (ie. disk) 
    - On-heap: subject to garbage collection
    - Off-heap: serialized data not subject to garbage collection
        Slightly slower than memory-based (have to be deserialized still)
        Still faster than disk-based
        Don’t waste GC’s time
        Java and Scala only!
- Cache: store in memory only
- Storage Levels: 
    - DISK_ONLY
    - MEMORY_ONLY
    - MEMORY_AND_DISK
    - Appending a number (ie. DISK_ONLY_2) adds that number of replicas
    - Appending SER (ie MEMORY_ONLY_SER) will serialize the data --> only in Java/Scala

# [RDDs](https://stackoverflow.com/questions/28981359/why-do-we-need-to-call-cache-or-persist-on-a-rdd)	
- Java/Scala only
- Nothing happens to data until actually perform an operation on it
    - Only references
- Linear situation: ie load file into rdd, perform a basic transformation, then count
    - Cache not needed
    - Data loaded to executors, transform, and count computed all in memory
- Non-linear: ie. load file, want to create a filtered dataset to work with
    - Cache before begin to branch
- Rule of thumb: cache when RDD branching out, used multiple times in a loop



# Pyspark Serializers
- Best practice to cache serialized objects
- Pickle Serializer 
    - More universal
- Marshall Serializer
    - Faster
- [TBD whether either of these actually offer an advantage over Kyros](https://stackoverflow.com/questions/36278574/do-you-benefit-from-the-kryo-serializer-when-you-use-pyspark)
